In [2]:
# libraries
import os
import pandas as pd
import altair as alt

In [7]:
"""
  

"""

def prepare_table(data, column_range):
    dataframe = pd.DataFrame()
    number_of_columns = len(column_range)
    for each_column in column_range:
        new_tobe_row = round(data[each_column].value_counts(normalize = True) * 100 ,0)
        #print(new_tobe_row)
        dataframe = pd.concat([dataframe, new_tobe_row], axis =1 )
    return dataframe.T

In [3]:
""""""
df = pd.read_excel('../data/Post-pandemic_travel_lodging-1.xlsx')

In [ ]:
# load the data
pc = pd.read_csv(os.path.join('..', 'data', 'quest.csv'))

In [4]:
# enabling altair rendering in notebook (needs to be explicitly set)
alt.renderers.enable('notebook')

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.


In [5]:
# define the response options (or column names)
col_vals = [
    '1', 
    '2', 
    '3', 
    '4', 
    '5',
    '6'
]

In [6]:
# define an extra 'start' column, the 'start' of every column
df['start'] = -df[col_vals[0:len(col_vals)//2]].sum(axis=1) -df[col_vals[len(col_vals)//2]]/2

KeyError: "None of [Index(['1', '2', '3'], dtype='object')] are in the [columns]"

In [ ]:
# properly 'melt' the dataframe for easier processing
df = pc[pc['subtable']=='all'].dropna(axis=1).melt(
    id_vars=[
        'question',
        'start'
    ], 
    value_vars=col_vals,
    var_name='variable'
)

In [ ]:
# define the offset, i.e. the start of the first column
offset = df[df['variable']==col_vals[0]]['start']

In [ ]:
# calculate start values for the other response options
for i in range(len(col_vals)-1):
    offset = offset + df.loc[df['variable'] == col_vals[i], 'value'].values
    df.loc[df['variable']==col_vals[i+1], 'start'] = offset.values 

In [ ]:
# add end of every column 
df['end'] = df['start'] + df['value']

In [ ]:
# define the colors, the default colors do not make sense here
color_scale = alt.Scale(
    domain=col_vals,
    range=["#c30d24", "#f3a583", "#cccccc", "#94c6da", "#1770ab"]
)

In [ ]:
# define the Y-axis
y_axis = alt.Axis(
    title='Question',
    offset=5,
    ticks=False,
    minExtent=60,
    domain=False
)

In [ ]:
# create the Altair chart
chart = alt.Chart(df).mark_bar().encode(
    x=alt.X('start:Q', title='Count'),
    x2='end:Q',
    y=alt.Y('question:N', axis=y_axis),
    color=alt.Color(
        'variable:N',
        legend=alt.Legend( title='Response'),
        scale=color_scale,
    )
)

In [ ]:
# display the chart
chart